In [1]:
#! lamin init --storage ./lamin-intro --schema bionty 

import scanpy as sc
from scprint import scPrint
from scdataloader import Preprocessor
from scdataloader.utils import load_genes
import numpy as np
import anndata as ad
from huggingface_hub import hf_hub_download

from scprint.tasks import Denoiser, withknn
from scprint.model.utils import downsample_profile
from scipy.stats import spearmanr
from anndata import AnnData

import seaborn as sns
%load_ext autoreload
%autoreload 2

import torch
torch.set_float32_matmul_precision('medium')

→ connected lamindb: jkobject/scprint2


In [2]:
#model_checkpoint_file = hf_hub_download(
#    repo_id="jkobject/scPRINT", filename=f"v2-medium.ckpt"
#)
#model_checkpoint_file = ../data/
model_checkpoint_file = '../../../wmpzj506.ckpt' 

In [14]:
m = torch.load(model_checkpoint_file)
m['hyper_parameters']['label_decoders'] = {k:{u: str(w) for u,w in v.items()} for k,v in m['hyper_parameters']['label_decoders'].items()}
m['hyper_parameters']['organisms'] = ["NCBITaxon:10090", "NCBITaxon:9606"]
torch.save(m, model_checkpoint_file)

In [3]:
model = scPrint.load_from_checkpoint(model_checkpoint_file, use_metacell_token=True, precpt_gene_emb=None)

FYI: scPrint is not attached to a `Trainer`.


In [ ]:
m = torch.load(model_checkpoint_file)
if "prenorm" in m['hyper_parameters']:
    m['hyper_parameters'].pop("prenorm")
    torch.save(m, model_checkpoint_file)
if "label_counts" in m['hyper_parameters']:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None, classes=m['hyper_parameters']['label_counts'])
else:
    model = scPrint.load_from_checkpoint(model_checkpoint_file, precpt_gene_emb=None)

In [ ]:
genes = load_genes(model.organisms)
set(model.genes) - set(genes.index)

/home/ml4ig1/Documents code/scDataLoader/scdataloader/utils.py:372: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  organismdf = pd.concat(organismdf)


set()

In [ ]:
#Get indices of genes that exist in genes. index
valid_indices = [i for i in range(len(model.genes)) if model.genes[i] in genes.index]
# Create-new •embedding-weights tensor with only valid genes new _weights -=-model.gene_encoder[01.embeddings.weight[valid_indices]
# Create- new-embedding layer with-correct size
new_embeddings = torch.nn.Embedding(len(valid_indices), model.gene_encoder[0].embeddings.embedding_dim, device=model.device)
#Copy the-weights-to-the new-embedding-layer new_embeddings.weight.data.= new_weights
#-Replace the old •embeddings with the new one
model.gene_encoder[0].embeddings = new_embeddings
model.genes = [i for i in model.genes if i in genes.index]

model = model.to(model.device)